In [1]:
%load_ext pycodestyle_magic
%pycodestyle_on

In [2]:
import doctest

In [3]:
import numbers


"""

>>> isinstance(True, int)
True
>>> isinstance(1, numbers.Integral)
True
>>> isinstance(5.3, numbers.Rational)
False
>>> x = int(1)
>>> isinstance(x, numbers.Real)
True
>>> isinstance(x, numbers.Number)
True
"""

doctest.testmod()

TestResults(failed=0, attempted=6)

In [4]:
import abc


class Tombola(abc.ABC):

    @abc.abstractmethod
    def load(self, iterable):
        pass

    @abc.abstractmethod
    def pick(self):
        pass

    def loaded(self):
        return bool(self.inspect())

    def inspect(self):
        items = []
        while True:
            try:
                items.append(self.pick())
            except LookupError:
                break

        self.load(items)
        return tuple(sorted(items))

In [5]:
class Fake(Tombola):

    def pick(self):
        return 13


f = Fake()

TypeError: Can't instantiate abstract class Fake with abstract method load

In [6]:
import random


class BingoCage(Tombola):

    def __init__(self, items):
        self._randomizer = random.SystemRandom()
        self._items = []
        self.load(items)

    def load(self, items):
        self._items.extend(items)
        self._randomizer.shuffle(self._items)

    def pick(self):
        cls = type(self)
        try:
            return self._items.pop()
        except IndexError:
            raise LookupError(f'pick from empty {cls.__name__}')

    def __call__(self):
        return self.pick()


"""

>>> b = BingoCage([1, 2, 3])
>>> b.loaded()
True
"""

doctest.testmod()

TestResults(failed=0, attempted=2)

In [7]:
import random


class LotteryBlower(Tombola):

    def __init__(self, iterable):
        self._balls = list(iterable)

    def load(self, iterable):
        self._balls.extend(iterable)

    def pick(self):
        cls = type(self)
        try:
            position = random.randrange(len(self._balls))
        except ValueError:
            raise LookupError(f'pick from empty {cls.__name__}')

        return self._balls.pop(position)

    def loaded(self):
        return bool(self._balls)

    def inspect(self):
        return tuple(sorted(self._balls))


lb = LotteryBlower([1, 2, 3, 4, 5])
while True:
    try:
        ball = lb.pick()
    except LookupError:
        break

    print(ball)

2
5
4
1
3


In [8]:
from random import randrange


@Tombola.register
class TomboList(list):

    def pick(self):
        cls = type(self)
        if self:
            position = randrange(len(self))
            return self.pop(position)
        else:
            raise LookupError(f'pop from empty {cls.__name__}')

    load = list.extend

    def loaded(self):
        return bool(self)

    def inspect(self):
        return tuple(sorted(self))


# Tombola.register(TomboList)
tl = TomboList([1, 2, 3])
issubclass(TomboList, Tombola), isinstance(tl, Tombola)

(True, True)

In [9]:

TEST_FILE = 'tombola_tests.rst'
TEST_MSG = '{0:16} {1.attempted:2} tests, {1.failed:2} failed - {2}'

VERBOSE = False

doctest_tmpl = ''

with open(TEST_FILE) as f:
    doctest_tmpl = f.read()


def test(cls, verbose=False):
    parser = doctest.DocTestParser()
    test = parser.get_doctest(
        doctest_tmpl,
        globs={'ConcreteTombola': cls},
        name=None,
        filename=TEST_FILE,
        lineno=0,
    )
    runner = doctest.DocTestRunner(
        optionflags=doctest.REPORT_ONLY_FIRST_FAILURE
    )
    runner.run(test)
    res = runner.summarize(verbose=verbose)
    tag = 'FAIL' if res.failed else 'OK'
    print(TEST_MSG.format(cls.__name__, res, tag))


# concrete subclasses
subclasses = Tombola.__subclasses__()

# virtual subclasses
_abc_registry = abc._get_dump(Tombola)[0]
subclasses += list(wref() for wref in _abc_registry)

for cls in subclasses:
    test(cls, VERBOSE)

**********************************************************************
File "tombola_tests.rst", line 10, in None
Failed example:
    globe = ConcreteTombola(balls)
Exception raised:
    Traceback (most recent call last):
      File "/usr/local/lib/python3.9/doctest.py", line 1336, in __run
        exec(compile(example.source, filename, "single",
      File "<doctest None[1]>", line 1, in <module>
    TypeError: Fake() takes no arguments
**********************************************************************
1 items had failures:
  18 of  24 in None
***Test Failed*** 18 failures.
Fake             24 tests, 18 failed - FAIL
BingoCage        24 tests,  0 failed - OK
LotteryBlower    24 tests,  0 failed - OK
TomboList        24 tests,  0 failed - OK
